In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import h5py
import sys
import os
import argparse
import tensorflow as tf
import numpy as np
import random
import math
from time import sleep
import glob

In [2]:
import import_ipynb
from params import model_params
p = model_params('config.ini')

importing Jupyter notebook from params.ipynb


In [3]:
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

config = ConfigProto()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2
# InteractiveSession(config=config)

In [4]:
def int64_feature(value):
    # 轉Int64資料為 tf.train.Feature 格式
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def bytes_feature(value):
    # 轉Bytes資料為 tf.train.Feature 格式
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [5]:
sr = 48000

In [6]:
def convert_to_TFRecord(h5_file_clean, h5_file_noisy, output_dir, n_tfrecords = 4):    
    # read h5
    with h5py.File(h5_file_clean, 'r') as hf:
        clean_data = hf['data'][:]
        n_samples = clean_data.shape[0]
        
    with h5py.File(h5_file_noisy, 'r') as hf:
        noisy_data = hf['data'][:]
        n_samples = noisy_data.shape[0]
    
    h5_file = h5_file_clean.split('.')[0].rstrip('_clean') + '.h5'
    #每個 TFR 文件放幾個檔案
    bestnum = math.ceil(n_samples/n_tfrecords)
    #第几个图片
    counter = 0
    #需要幾個 TFR
    if n_samples%bestnum == 0:
        num_TFR_file = n_samples//bestnum
    else:
        num_TFR_file = n_samples//bestnum+1

    #creat output directory
    if not os.path.exists(output_dir):
        os.umask(0)
        os.makedirs(output_dir, mode=0o755)    
    
    # for loop to generate tfrecords
    for k in range(num_TFR_file):
        filename = ((h5_file.split('/')[-1]).split('.')[0] + '-{}-of-{}.tfrecord'.format(k+1, num_TFR_file))
        TFR_gen_path = os.path.join(output_dir, filename)

        # declare TFwriter
        TFWriter = tf.io.TFRecordWriter(TFR_gen_path)

        print('\nStart generate {}'.format(filename))
        for i in range(bestnum):#//p.length_sec):
            try:
                start_idx = bestnum * k + i * 1
                end_idx = bestnum * k + (i+1) * 1
                
                h5_X = noisy_data[start_idx: end_idx,:]
                h5_Y = clean_data[start_idx: end_idx,:]
                
                h5_X_flatten = h5_X.flatten()
                h5_Y_flatten = h5_Y.flatten()
                
#                 print(h5_X_flatten.shape)
#                 for j in range(3):
#                     # 將 image and label 合併成 tf.train.Features
#                     ftrs = tf.train.Features(
#                             feature={'X': float_feature(h5_X_flatten[j*sr:(j+1)*sr]),
#                                      'Y': float_feature(h5_Y_flatten[j*sr:(j+1)*sr])})

#                     # 將 tf.train.Features 轉成 tf.train.Example
#                     example = tf.train.Example(features=ftrs)

#                     # 將 tf.train.Example 寫成 tfRecord 格式
#                     TFWriter.write(example.SerializeToString())
                # 將 image and label 合併成 tf.train.Features
                ftrs = tf.train.Features(
                        feature={'X': float_feature(h5_X_flatten),
                                 'Y': float_feature(h5_Y_flatten)})

                # 將 tf.train.Features 轉成 tf.train.Example
                example = tf.train.Example(features=ftrs)

                # 將 tf.train.Example 寫成 tfRecord 格式
                TFWriter.write(example.SerializeToString())
                
                counter = counter + 1
                                   
                if counter%30000 == 0:
                    print('generating {} samples'.format(counter))
                    sleep(0.1)
                
                if counter == n_samples:
                    print('counter reach {}'.format(n_samples))
                    break
                    
            except IOError as e:
                print('Skip!\n')
                

        TFWriter.close()
    print('Transform done!')
    print('total {} data processed'.format(counter))

In [7]:
# def convert_to_TFRecord(h5_file, output_dir, n_tfrecords = 4):    
#     # read h5
#     with h5py.File(h5_file, 'r') as hf:
#         data = hf['data'][:]
#         n_samples = data.shape[0]
    
#     print(h5_file,':',data.shape)
#     #每個 TFR 文件放幾個檔案
#     bestnum = math.ceil(n_samples/n_tfrecords)
#     print(bestnum)
#     #第几个图片
#     counter = 0
#     #需要幾個 TFR
#     if n_samples%bestnum == 0:
#         num_TFR_file = n_samples//bestnum
#     else:
#         num_TFR_file = n_samples//bestnum+1

#     #creat output directory
#     if not os.path.exists(output_dir):
#         os.umask(0)
#         os.makedirs(output_dir, mode=0o755)    
    
#     # for loop to generate tfrecords
#     for k in range(num_TFR_file):
#         filename = ((h5_file.split('/')[-1]).split('.')[0] + '-{}-of-{}.tfrecord'.format(k+1, num_TFR_file))
#         TFR_gen_path = os.path.join(output_dir, filename)

#         # declare TFwriter
#         TFWriter = tf.io.TFRecordWriter(TFR_gen_path)

#         print('\nStart generate {}'.format(filename))
#         for i in range(bestnum//p.length_sec//100):
#             try:                
#                 h5_X = data[(k*bestnum)+i*p.length_sec*100:(k*bestnum)+(i+1)*p.length_sec*100,:]
#                 print(h5_file,':',h5_X.shape)
#                 print((k*bestnum)+i*p.length_sec*100)
#                 # 將 image and label 合併成 tf.train.Features
#                 ftrs = tf.train.Features(
#                         feature={'X': float_feature(h5_X.flatten())})
                                

#                 # 將 tf.train.Features 轉成 tf.train.Example
#                 example = tf.train.Example(features=ftrs)

#                 # 將 tf.train.Example 寫成 tfRecord 格式
#                 TFWriter.write(example.SerializeToString())
                
#                 counter = counter + 1
                                   
#                 if counter%30000 == 0:
#                     print('generating {} samples'.format(counter))
#                     sleep(0.1)
                
#                 if counter == n_samples:
#                     print('counter reach {}'.format(n_samples))
#                     break
                    
#             except IOError as e:
#                 print('Skip!\n')
                

#         TFWriter.close()
#     print('Transform done!')
#     print('total {} data processed'.format(counter))

In [8]:
# file directory
TFR_ROOT ='/home3/user/myhsueh/h5_dataset/'
print("file directory exist: {}".format(os.path.exists(TFR_ROOT)))
      
# specify the filename      
TFR_FILE_PATH = os.path.join(TFR_ROOT, '*_clean.h5')
# TFR_FILE_PATH = os.path.join(TFR_ROOT, '*.h5')

# use glob module to grab all the files in a list
filenames = glob.glob(TFR_FILE_PATH)
# filenames = filenames[:7]
print(filenames)
print(len(filenames))

file directory exist: True
['/home3/user/myhsueh/h5_dataset/TD_26_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_31_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_74KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_22_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_20_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_24_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_85KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_41_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_71KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_78KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_23_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_19_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_81KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_47_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_88KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_82KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_87KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_86KB_clean.h5', '/home3/user/myhsueh/h5_dataset/TD_44_clean.h5', '/home3/user/myhsueh/h5

In [9]:
def convert(file_list, dir_path = '/home2/user/myhsueh/dataset/tfrecord/no_rir/training/'):
    for h5_path_clean in file_list:
        path, h5_file_clean = os.path.split(h5_path_clean)
        filename_split = h5_file_clean.split('.')
        h5_file_noisy =  filename_split[0].strip('clean') + 'noisy.' + filename_split[1]
        h5_path_noisy = os.path.join(path, h5_file_noisy)
        convert_to_TFRecord(h5_path_clean, h5_path_noisy, dir_path)

In [10]:
# import multiprocessing as mp
# threads = 20

# num_of_h5 = len(filenames)
# # num_of_each_thread = int(num_of_h5*0.8) // threads
# num_of_each_thread = int(num_of_h5) // threads
# train_list = [filenames[i*num_of_each_thread:(i+1)*num_of_each_thread] for i in range(threads)]

In [11]:
# pool = mp.Pool(processes=threads)
# pool.map(convert, train_list)

In [12]:
filenames = ['/home3/user/myhsueh/h5_dataset/TD_3_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_11_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_13_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_14_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_16_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_34_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_38_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_39_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_43_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_50_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_62_clean.h5', 
             '/home3/user/myhsueh/h5_dataset/TD_63_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_79KB_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_83KB_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_84KB_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_91KB_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_92KB_clean.h5',
             '/home3/user/myhsueh/h5_dataset/TD_98KB_clean.h5']

In [14]:
for i in filenames:
    convert([i])


Start generate TD_3-1-of-4.tfrecord

Start generate TD_3-2-of-4.tfrecord

Start generate TD_3-3-of-4.tfrecord

Start generate TD_3-4-of-4.tfrecord
counter reach 2000
Transform done!
total 2000 data processed

Start generate TD_11-1-of-4.tfrecord

Start generate TD_11-2-of-4.tfrecord

Start generate TD_11-3-of-4.tfrecord

Start generate TD_11-4-of-4.tfrecord
counter reach 2000
Transform done!
total 2000 data processed

Start generate TD_13-1-of-4.tfrecord

Start generate TD_13-2-of-4.tfrecord

Start generate TD_13-3-of-4.tfrecord

Start generate TD_13-4-of-4.tfrecord
counter reach 2000
Transform done!
total 2000 data processed

Start generate TD_14-1-of-4.tfrecord

Start generate TD_14-2-of-4.tfrecord

Start generate TD_14-3-of-4.tfrecord

Start generate TD_14-4-of-4.tfrecord
counter reach 2000
Transform done!
total 2000 data processed

Start generate TD_16-1-of-4.tfrecord

Start generate TD_16-2-of-4.tfrecord

Start generate TD_16-3-of-4.tfrecord

Start generate TD_16-4-of-4.tfrecord
